In [23]:
import sys
import os  
import traceback

RECORDS_PATH = '../input/tfrecords'

!ln -s ../input/feedback ./src

if os.path.exists('/kaggle'):
  sys.path.append('/kaggle/input/pikachu/utils')
  sys.path.append('/kaggle/input/pikachu/third')
  sys.path.append('.')
  from kaggle_datasets import KaggleDatasets
#   try:
#     RECORDS_PATH = KaggleDatasets().get_gcs_path('toxic-multi-tfrecords') + '/tfrecords/tfrecords'
#     os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#   except Exception:
#     print(traceback.format_exc())
#     pass
!ls ../input
RECORDS_PATH
!ls .

In [24]:
!pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream/ 

In [25]:
import tensorflow as tf
import torch
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer
from datasets import Dataset
from src import config
from src.torch_model import Model as TorchModel
from src.model import Model
from src.util import get_preds
import melt as mt
import numpy as np
from gezi import tqdm
import gezi
import husky
import lele

In [26]:
model_dir = '../input/feedback-model'
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
FLAGS.wandb = False
config.init_()
FLAGS.backbone = '../input/' + FLAGS.backbone.split('/')[-1]
ic(FLAGS.backbone)
# mt.init()
MAX_LEN = FLAGS.max_len
tokenizer = AutoTokenizer.from_pretrained(FLAGS.backbone, add_prefix_space=True)
if not FLAGS.torch:
  model = Model().build_model()
  model.load_weights(f'{model_dir}/model2.h5')
else:
  model = TorchModel()
  lele.load(model, f'{model_dir}/model.pt')
open(f'{model_dir}/path.txt').readline().strip()

In [ ]:
files = os.listdir('../input/feedback-prize-2021/test')
TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
if len(TEST_IDS) < 10:
  for _ in range(6):
    TEST_IDS += TEST_IDS
print('There are',len(TEST_IDS),'test texts.')

In [27]:
# CONVERT TEST TEXT TO TOKENS
input_ids = np.zeros((len(TEST_IDS), MAX_LEN), dtype='int32')
attention_mask = np.zeros((len(TEST_IDS), MAX_LEN), dtype='int32')
word_ids_list = []
for id_num in tqdm(range(len(TEST_IDS))):
  n = TEST_IDS[id_num]
  name = f'../input/feedback-prize-2021/test/{n}.txt'
  txt = open(name, 'r').read()
  encoding = tokenizer(txt.split(),
                        is_split_into_words=True,
                        padding='max_length', 
                        truncation=True, 
                        max_length=MAX_LEN)
  input_ids[id_num,] = encoding['input_ids']
  attention_mask[id_num,] = encoding['attention_mask']
  word_ids = encoding.word_ids()
  word_ids = [-1 if x is None else x for x in word_ids]
  word_ids_list.append(word_ids)
inputs = {
   'input_ids': input_ids,
   'attention_mask': attention_mask
   }

In [ ]:
# TODO for longformer large seems torch can use much larger bs for inference? for torch large+32 use 13.9G 
BATCH_SIZE = 8 if not FLAGS.torch else 32 
# INFER TEST TEXTS
if not FLAGS.torch:
  inputs = {
   'tokens': input_ids,
   'attention': attention_mask
   }
  dataset = tf.data.Dataset.from_tensor_slices(inputs).batch(BATCH_SIZE)
  strategy = mt.distributed.get_strategy()
  with strategy.scope(): 
    p = model.predict(dataset, callbacks=[husky.TQDMProgressBar()])
else:
  dataset = Dataset.from_dict(inputs)
  dataset.set_format(type='torch', device='cuda')
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)
  p = lele.predict(model, dataloader)

In [ ]:
preds = np.argmax(p['pred'],axis=-1)
scores = gezi.softmax(p['pred'], axis=-1)
preds

In [ ]:
if 'start_logits' in p:
  preds = (preds, gezi.softmax(p['start_logits'], axis=-1))
df = get_preds(TEST_IDS, preds, word_ids_list, scores)
df

In [ ]:
df.to_csv('submission.csv',index=False)